In [1]:
import sys
sys.path.append('/home/gbortolai/Thesis/')

In [1]:
import torch
import numpy as np
import ot
from multiprocessing import Pool
import time

In [2]:
#GPU
# device = "cuda:0" if torch.cuda.is_available() else "cpu"
device = "cpu"
print(f"Using {device} device")

Using cpu device


In [3]:
def sliced_wasserstein_dist(a,b):
    
    source = a[torch.nonzero(a, as_tuple = False)[:,0], torch.nonzero(a, as_tuple = False)[:,1]].view(torch.nonzero(a, as_tuple = False).size(0)).float()
    target = b[torch.nonzero(b, as_tuple = False)[:,0], torch.nonzero(b, as_tuple = False)[:,1]].view(torch.nonzero(b, as_tuple = False).size(0)).float()
    
    source = torch.unsqueeze(source, dim = 1)
    target = torch.unsqueeze(target, dim = 1)
    
    source = source.to(device)
    target = target.to(device)
    
    W = ot.sliced.sliced_wasserstein_distance(source, target, p = 2).to(device)
    
    return W

In [ ]:
def fun(data_source, data_target, ind1, ind2):
    
    SW = torch.zeros(data_source.size(0), data_target.size(0))
    
    if ind1 == ind2:
        for i in range(data_source.size(0)):
            for j in filter(lambda h: h>i, range(data_target.size(0))):

                SW[i][j] = sliced_wasserstein_dist(data_source[i], data_target[j])
                
    elif ind2 > ind1:
        
        for i in range(data_source.size(0)):
            for j in range(data_target.size(0)):
                
                SW[i][j] = sliced_wasserstein_dist(data_source[i], data_target[j])
            
    return SW

In [ ]:
def iterable(data, processes):
    
    data = torch.stack(torch.chunk(data, processes, dim = 0), dim = 0)
    
    for i in range(data.size(0)):
        for j in range(data.size(0)):
            
            yield([data[i], data[j], i, j])

In [4]:
data, targets = torch.load('data/Jets/dataset_train')
data = data.to(device)

In [ ]:
# size = data.size(0)

size = 100

processes = 2

Sliced_Wasserstein_dist = torch.zeros(size, size)

ite = iterable(data[:size], processes)

st = time.time()
with Pool(processes = processes) as p:

    Sliced_Wasserstein_dist = torch.cat(torch.chunk(torch.cat(p.starmap(fun, ite), dim = 1), processes, dim = 1), dim = 0)
    
et = time.time()
print(et - st)

print(Sliced_Wasserstein_dist)

In [7]:
# #Wasserstein distance 
size = data.size(0)
# size = 200

Sliced_Wasserstein_dist=torch.zeros(size, size, dtype = torch.float).to(device)

for i in range (size):
    for j in filter(lambda h: h>i, range (size)):
        Sliced_Wasserstein_dist[i][j] = sliced_wasserstein_dist(data[i], data[j]).to(device)
    if i%5 == 0:
        print(f'Rows done: {(i/size)*100} %')

Rows done: 0.0 %
Rows done: 2.5 %
Rows done: 5.0 %
Rows done: 7.5 %
Rows done: 10.0 %
Rows done: 12.5 %
Rows done: 15.0 %
Rows done: 17.5 %
Rows done: 20.0 %
Rows done: 22.5 %
Rows done: 25.0 %
Rows done: 27.500000000000004 %
Rows done: 30.0 %
Rows done: 32.5 %
Rows done: 35.0 %
Rows done: 37.5 %
Rows done: 40.0 %
Rows done: 42.5 %
Rows done: 45.0 %
Rows done: 47.5 %
Rows done: 50.0 %
Rows done: 52.5 %
Rows done: 55.00000000000001 %
Rows done: 57.49999999999999 %
Rows done: 60.0 %
Rows done: 62.5 %
Rows done: 65.0 %
Rows done: 67.5 %
Rows done: 70.0 %
Rows done: 72.5 %
Rows done: 75.0 %
Rows done: 77.5 %
Rows done: 80.0 %
Rows done: 82.5 %
Rows done: 85.0 %
Rows done: 87.5 %
Rows done: 90.0 %
Rows done: 92.5 %
Rows done: 95.0 %
Rows done: 97.5 %
29.99451780319214


In [21]:
torch.save([Sliced_Wasserstein_dist, targets],'data/Jets/Sliced_Wasserstein_dist_train_simple_s='+str(size))